<a href="https://colab.research.google.com/github/vuongvmu/GCL_DemoCode/blob/main/DecodeVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import asyncio
import av
import cv2
import io
from multiprocessing import Process, Queue, Event
import time
import websockets
import socket
import time
import base64
import os

def decode(terminated, data_queue):
     memBuff=io.BytesIO()
     cur_pos=0
     frames_buffer=[]
     pts_offset=None
     got_key_frame = False
     codec = av.codec.CodecContext.create("h264", 'r')

     while not terminated.is_set():
          try:
               data=data_queue.get_nowait() #Lấy tất cả buff trong queue
          except:
               time.sleep(0.01)
               continue
          memBuff.write(data)# lưu data vào memory
          memBuff.seek(cur_pos)
          if cur_pos==0:
               container =av.open(memBuff,mode='r')
          cur_pos+=len(data)
          dts=None
          for packet in container.demux():
               if packet.size == 0:
                   continue
               dts = packet.dts #decoding timestamp
               if pts_offset is None:
                   pts_offset = packet.pts #presentation timestamp
               if not got_key_frame and packet.is_keyframe:
                   got_key_frame = True
               if data_queue.qsize() > 8 and not packet.is_keyframe:
                   got_key_frame = False
                   continue
               if not got_key_frame:
                   continue
               frames = codec.decode(packet)# decode video
               for frame in frames:
                    data_queue.put(frame.to_ndarray(format='bgr24'))# covert frame to ndarray
          if dts is not None:
              container.seek(25000)
          memBuff.seek(cur_pos)
     terminated.set()

In [ ]:
#Start thread
data_queue = Queue()
terminated = Event()

.... thread.....
